In [ ]:
import pandas as pd
import numpy as np

# Create a sample DataFrame
df = pd.DataFrame({
    'A': ['foo', 'foo', 'foo', 'bar', 'bar', 'bar'],
    'B': ['one', 'one', 'two', 'two', 'one', 'one'],
    'C': ['small', 'large', 'large', 'small', 'small', 'large'],
    'D': [1, 2, 2, 3, 3, 4],
    'E': [2, 4, 5, 5, 6, 6]
})

df

,A,B,C,D,E
0,foo,one,small,1,2
1,foo,one,large,2,4
2,foo,two,large,2,5
3,bar,two,small,3,5
4,bar,one,small,3,6
5,bar,one,large,4,6


In [ ]:
pivot_table = pd.pivot_table(df, values=['D','E'], index=['A', 'B'], columns=['C'], aggfunc="sum", fill_value=0)

pivot_table

D           E      
C       large small large small
A   B                          
bar one     4     3     6     6
    two     0     3     0     5
foo one     2     1     4     2
    two     2     0     5     0

In [ ]:
# Example 1: Sales Data Analysis
# Create a sample DataFrame
sales_data = pd.DataFrame({
    'Date': pd.date_range(start='2023-01-01', end='2023-12-31', freq='D'),
    'Product': np.random.choice(['A', 'B', 'C'], size=365),
    'Region': np.random.choice(['North', 'South', 'East', 'West'], size=365),
    'Sales': np.random.randint(100, 1000, size=365),
    'Units': np.random.randint(1, 50, size=365)
})

sales_data.head()

,Date,Product,Region,Sales,Units
0,2023-01-01,B,North,955,45
1,2023-01-02,B,North,156,28
2,2023-01-03,A,North,725,47
3,2023-01-04,C,North,117,1
4,2023-01-05,A,North,114,36


In [ ]:
# monthly sales by product

monthly_sales_by_product = pd.pivot_table(sales_data, values='Sales', index= sales_data['Date'].dt.to_period('M'), columns='Product', aggfunc='sum', fill_value=0)

monthly_sales_by_product

Product,A,B,C
Date,,,
2023-01,5250,5175,4212
2023-02,3867,9029,3975
2023-03,6489,5002,4402
2023-04,5613,4248,6449
2023-05,6304,6958,5154
2023-06,3907,6135,6854
2023-07,3947,4437,8007
2023-08,4019,4015,8589
2023-09,3552,3145,7741


In [ ]:
# Example 2: Custom Aggregation Function
def range_metric(x):
    return x.max() - x.min()

# Pivot table with custom aggregation and margins
# Create the pivot table without margins first
sales_summary_temp = pd.pivot_table(sales_data,
                                     values=['Sales', 'Units'],
                                     index='Region',
                                     columns='Product',
                                     aggfunc={'Sales': ["mean","sum"], 'Units': ["mean", range_metric]},
                                     fill_value=0)

# Rename the columns to avoid conflicts
sales_summary_temp.columns = ['_'.join(map(str, col)).strip() for col in sales_summary_temp.columns.values]

# Now add margins
sales_summary = sales_summary_temp.assign(Total=sales_summary_temp.sum(axis=1))
sales_summary.loc['Total'] = sales_summary.sum(numeric_only=True)

print("\nSales Summary with Custom Aggregation and Margins:")
print(sales_summary)


Sales Summary with Custom Aggregation and Margins:
        Sales_mean_A  Sales_mean_B  Sales_mean_C  Sales_sum_A  Sales_sum_B  \
Region                                                                       
East      494.931034    582.125000    569.769231      14353.0      18628.0   
North     530.464286    544.653846    495.343750      14853.0      14161.0   
South     652.500000    520.781250    587.444444      19575.0      16665.0   
West      524.586207    549.409091    526.333333      15213.0      12087.0   
Total    2202.481527   2196.969187   2178.890759      63994.0      61541.0   

        Sales_sum_C  Units_mean_A  Units_mean_B  Units_mean_C  \
Region                                                          
East        22221.0     26.034483     23.718750     28.538462   
North       15851.0     31.035714     29.115385     21.937500   
South       15861.0     25.400000     20.593750     21.222222   
West        20527.0     25.275862     21.590909     23.333333   
Total      

In [ ]:
# Example 3: Customer Segmentation
# Create a sample DataFrame
customer_data = pd.DataFrame({
    'Customer_ID': range(1, 101),
    'Age_Group': np.random.choice(['18-25', '26-35', '36-45', '46+'], size=100),
    'Gender': np.random.choice(['Male', 'Female'], size=100),
    'Product_Category': np.random.choice(['Electronics', 'Clothing', 'Books', 'Home'], size=100),
    'Purchase_Amount': np.random.randint(10, 500, size=100)
})

# Pivot table: Average purchase amount by age group and gender for each product category
customer_segments = pd.pivot_table(customer_data,
                                   values='Purchase_Amount',
                                   index=['Age_Group', 'Gender'],
                                   columns='Product_Category',
                                   aggfunc='mean',
                                   fill_value=0,
                                   margins=True)

print("\nCustomer Segmentation - Average Purchase Amount:")
print(customer_segments)






Customer Segmentation - Average Purchase Amount:
Product_Category       Books    Clothing  Electronics        Home         All
Age_Group Gender                                                             
18-25     Female    0.000000  225.666667   274.000000  217.750000  237.214286
          Male    276.500000  210.666667   345.000000  262.666667  266.300000
26-35     Female  222.400000  226.000000   427.500000  213.000000  249.066667
          Male    317.000000  283.250000   241.428571  347.666667  281.250000
36-45     Female  195.666667  283.333333   196.750000  424.000000  249.857143
          Male    249.000000  185.666667   320.500000    0.000000  244.300000
46+       Female  347.333333    0.000000   318.500000    0.000000  330.857143
          Male    387.000000  225.600000   249.166667  210.500000  245.071429
All               264.761905  238.562500   275.096774  261.625000  259.080000


In [ ]:
# Example 4: Multi-level Pivot Table
# Create a sample DataFrame
company_data = pd.DataFrame({
    'Year': np.repeat(range(2020, 2024), 80),
    'Quarter': np.tile(np.repeat(['Q1', 'Q2', 'Q3', 'Q4'], 20), 4),
    'Department': np.tile(np.repeat(['Sales', 'Marketing', 'IT', 'HR'], 5), 16),
    'Employee': np.tile(range(1, 6), 64),
    'Performance': np.random.randint(1, 6, size=320),
    'Satisfaction': np.random.randint(1, 6, size=320)
})

# Multi-level pivot table: Average performance and satisfaction by year, quarter and department
company_performance = pd.pivot_table(company_data,
                                     values=['Performance', 'Satisfaction'],
                                     index=['Year', 'Quarter'],
                                     columns='Department',
                                     aggfunc='mean',
                                     fill_value=0)

print("\nCompany Performance and Satisfaction:")
print(company_performance)


Company Performance and Satisfaction:
             Performance                      Satisfaction                 \
Department            HR   IT Marketing Sales           HR   IT Marketing   
Year Quarter                                                                
2020 Q1              2.6  2.0       2.6   2.8          3.0  3.0       3.2   
     Q2              2.8  2.0       3.8   3.4          3.0  3.6       3.4   
     Q3              3.8  3.2       3.0   3.8          2.8  3.6       2.2   
     Q4              4.2  3.0       2.2   3.0          3.0  3.4       4.0   
2021 Q1              4.0  3.4       1.8   3.2          3.4  3.6       3.2   
     Q2              3.0  3.4       2.2   3.2          3.2  3.0       3.0   
     Q3              4.2  2.6       3.4   2.6          2.6  3.4       4.0   
     Q4              4.0  2.6       3.2   3.4          2.4  1.6       2.8   
2022 Q1              2.6  2.8       2.8   2.0          3.0  3.4       2.6   
     Q2              3.6  3.6       3

## Advanced Example

In [ ]:
# Sample DataFrame
df = pd.DataFrame({
    'Date': pd.date_range(start='2023-01-01', periods=100),
    'Product': np.random.choice(['A', 'B', 'C'], 100),
    'Category': np.random.choice(['X', 'Y'], 100),
    'Sales': np.random.randint(100, 1000, 100),
    'Quantity': np.random.randint(1, 50, 100),
    'Returns': np.random.randint(0, 10, 100)
})

# 1. Multiple Aggregation Functions
multi_agg = pd.pivot_table(df,
                           values=['Sales', 'Quantity', 'Returns'],
                           index=['Product', 'Category'],
                           aggfunc={'Sales': ['sum', 'mean'],  # Changed to strings
                                    'Quantity': ['sum', 'max'],
                                    'Returns': ['sum', lambda x: x.max() - x.min()]},
                           fill_value=0)

print("1. Multiple Aggregation Functions:")
print(multi_agg)

# 2. Calculated Fields
def profit_calc(data):
    return data['Sales'].sum() - (data['Quantity'].sum() * 50)  # Assuming cost of 50 per unit

calc_fields = pd.pivot_table(df,
                             values=['Sales', 'Quantity'],
                             index='Product',
                             columns='Category',
                             aggfunc={'Sales': 'sum', 'Quantity': 'sum'},  # Changed to strings
                             margins=True,
                             margins_name='Total')

calc_fields['Profit'] = calc_fields.apply(profit_calc, axis=1)

print("\n2. Calculated Fields:")
print(calc_fields)

# 3. Time-based Grouping
time_group = pd.pivot_table(df,
                            values='Sales',
                            index=[df['Date'].dt.year, df['Date'].dt.month],
                            columns='Product',
                            aggfunc='sum',  # Changed to string
                            fill_value=0)

print("\n3. Time-based Grouping:")
print(time_group)

# 4. Hierarchical Indexing and Sorting
hier_index = pd.pivot_table(df,
                            values=['Sales', 'Returns'],
                            index=['Category', 'Product'],
                            aggfunc={'Sales': 'sum', 'Returns': 'mean'},  # Changed to strings
                            fill_value=0)

# Corrected sorting
hier_index_sorted = hier_index.sort_values(by='Sales',  # Use just 'Sales' as it is now a string
                                           ascending=False)

print("\n4. Hierarchical Indexing and Sorting:")
print(hier_index_sorted)

# 5. Custom Display Formatting
def format_sales(x):
    return f"${x:,.2f}"

custom_format = pd.pivot_table(df,
                               values=['Sales', 'Quantity'],
                               index='Product',
                               columns='Category',
                               aggfunc={'Sales': 'sum', 'Quantity': 'mean'},  # Changed to strings
                               fill_value=0)

custom_format['Sales'] = custom_format['Sales'].applymap(format_sales)
custom_format['Quantity'] = custom_format['Quantity'].applymap(lambda x: f"{x:.1f}")

print("\n5. Custom Display Formatting:")
print(custom_format)

In [37]:
import cProfile
import pstats


Best Practices and Performance Considerations for pandas pivot_table()
When working with pivot_table() in pandas, especially on large datasets, it's important to consider both best practices for clarity and techniques for optimizing performance. Here are some key points to keep in mind:

1. D**ata Preparation** <br>
Clean your data: Remove or handle missing values, duplicates and outliers before creating pivot tables.
Ensure correct data types: Convert columns to appropriate dtypes (e.g., categorical for low-cardinality columns) to improve memory usage and performance.

2. **Efficient Indexing** <br>
Use hierarchical indexing judiciously: While powerful, excessive levels can make the result hard to interpret.
Consider using .query() or boolean indexing for filtering before pivoting, especially for large datasets.


3. **Aggregation Functions** <br>
Use built-in functions when possible: NumPy and pandas built-in functions are often more optimized than custom functions.
For custom functions, consider using numba for performance: If you need a custom aggregation function, Numba can significantly speed up execution.


4. **Memory Management** <br>
Avoid duplicating data: Use observed=True for categorical columns to only show observed categories, reducing memory usage.
Use fill_value judiciously: Setting fill_value=0 can increase memory usage for sparse data.


5. **Alternative Methods** <br>
Consider alternatives for very large datasets: For extremely large datasets, consider using groupby() operations or database-level pivoting before loading into pandas.

6. **Readability and Maintainability** <br>
Use descriptive names for aggregation functions: When using multiple or custom aggregations, use a dict with descriptive names.
Document complex pivot operations: Add comments explaining the purpose and structure of complex pivot tables.

7. **Performance Monitoring** <br>
Profile your code: Use tools like cProfile or line_profiler to identify bottlenecks in your pivot table operations.
Monitor memory usage: Tools like memory_profiler can help identify memory-intensive operations.

### Side Notes Knowing How this code works

In [20]:
pd.date_range(start='2023-01-01', periods=5)

DatetimeIndex(['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04',
               '2023-01-05'],
              dtype='datetime64[ns]', freq='D')

In [22]:
pd.date_range(start='2023-01-01', end='2023-01-31',freq='h')[:10]

DatetimeIndex(['2023-01-01 00:00:00', '2023-01-01 01:00:00',
               '2023-01-01 02:00:00', '2023-01-01 03:00:00',
               '2023-01-01 04:00:00', '2023-01-01 05:00:00',
               '2023-01-01 06:00:00', '2023-01-01 07:00:00',
               '2023-01-01 08:00:00', '2023-01-01 09:00:00'],
              dtype='datetime64[ns]', freq='h')

In [23]:
pd.date_range(start='2023-01-01', end='2023-01-31',freq='h').to_period('d')[:10]

PeriodIndex(['2023-01-01', '2023-01-01', '2023-01-01', '2023-01-01',
             '2023-01-01', '2023-01-01', '2023-01-01', '2023-01-01',
             '2023-01-01', '2023-01-01'],
            dtype='period[D]')

In [24]:
np.repeat(range(2020, 2024), 5) # repeats each value in range 5 times

array([2020, 2020, 2020, 2020, 2020, 2021, 2021, 2021, 2021, 2021, 2022,
       2022, 2022, 2022, 2022, 2023, 2023, 2023, 2023, 2023])

In [25]:
np.tile(np.repeat(range(2020, 2024), 5) , 2) # repeats the cycle

array([2020, 2020, 2020, 2020, 2020, 2021, 2021, 2021, 2021, 2021, 2022,
       2022, 2022, 2022, 2022, 2023, 2023, 2023, 2023, 2023, 2020, 2020,
       2020, 2020, 2020, 2021, 2021, 2021, 2021, 2021, 2022, 2022, 2022,
       2022, 2022, 2023, 2023, 2023, 2023, 2023])

In [31]:
!pip install snakeviz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 5.5 MB/s eta 0:00:00


In [36]:

def test_func():
    company_data = pd.DataFrame({
    'Year': np.repeat(range(2020, 2024), 80),
    'Quarter': np.tile(np.repeat(['Q1', 'Q2', 'Q3', 'Q4'], 20), 4),
    'Department': np.tile(np.repeat(['Sales', 'Marketing', 'IT', 'HR'], 5), 16),
    'Employee': np.tile(range(1, 6), 64),
    'Performance': np.random.randint(1, 6, size=320),
    'Satisfaction': np.random.randint(1, 6, size=320)
    })

# Multi-level pivot table: Average performance and satisfaction by year, quarter and department
    company_performance = pd.pivot_table(company_data,
                                     values=['Performance', 'Satisfaction'],
                                     index=['Year', 'Quarter'],
                                     columns='Department',
                                     aggfunc='mean',
                                     fill_value=0)

In [41]:

cProfile.run('test_func()', 'Bonga')
p = pstats.Stats('Bonga')
p.sort_stats('ncalls','cumulative').print_stats(5)


Mon Nov  4 12:02:39 2024    Bonga

         6284 function calls (6164 primitive calls) in 0.019 seconds

   Ordered by: call count, cumulative time
   List reduced from 600 to 5 due to restriction <5>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
1222/1217    0.001    0.000    0.001    0.000 {built-in method builtins.isinstance}
  472/362    0.000    0.000    0.001    0.000 {built-in method builtins.len}
      296    0.000    0.000    0.000    0.000 {built-in method builtins.getattr}
      185    0.000    0.000    0.000    0.000 /usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/generic.py:42(_instancecheck)
      185    0.000    0.000    0.000    0.000 /usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/generic.py:37(_check)


